In [1]:
import collections
import os
import shutil
import tqdm

import numpy as np
import PIL.Image
import torch
import torchvision

from IPython.display import display

from pprint import pprint
from k12libs.utils.nb_easy import k12ai_get_top_dir
from k12libs.utils.nb_easy import k12ai_print

# 检查PyTorch版本

In [2]:
print(torch.__version__)               # PyTorch version
print(torch.version.cuda)              # Corresponding CUDA version
print(torch.backends.cudnn.version())  # Corresponding cuDNN version
print(torch.cuda.get_device_name(0))   # GPU type
print(torch.cuda.is_available())

1.5.0.dev20200319
10.1
7603
Tesla P40
True


# 固定随机种子

In [3]:
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

# 设置为cuDNN benchmark模式

In [4]:
# Benchmark模式会提升计算速度，但是由于计算中有随机性，每次网络前馈结果略有差异。
torch.backends.cudnn.benchmark = True
# 如果想要避免这种结果波动
torch.backends.cudnn.deterministic = True

# 创建与复制

## 由numpy构建,采用深度复制

In [5]:
np_a = np.array([1,2,3])
tensor_a = torch.tensor(np_a)
np_a, tensor_a

(array([1, 2, 3]), tensor([1, 2, 3]))

In [6]:
tensor_a[0] = -1
np_a, tensor_a

(array([1, 2, 3]), tensor([-1,  2,  3]))

## copy构造(不建议使用)

In [7]:
tensor_a = torch.tensor([1.,2.,3.], requires_grad=True)
tensor_t = torch.tensor(tensor_a)
tensor_t[0] = -1.
tensor_a, tensor_t, tensor_a == tensor_t, id(tensor_a), id(tensor_t) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


(tensor([1., 2., 3.], requires_grad=True),
 tensor([-1.,  2.,  3.]),
 tensor([False,  True,  True]),
 140300163941288,
 140300163941360)

In [8]:
tensor_t = tensor_a.clone()
tensor_t[0] = -1.0
tensor_a, tensor_t, tensor_a == tensor_t, id(tensor_a), id(tensor_t)

(tensor([1., 2., 3.], requires_grad=True),
 tensor([-1.,  2.,  3.], grad_fn=<CopySlices>),
 tensor([False,  True,  True]),
 140300163941288,
 140300163943880)

## tensor.data 不建议使用(共享储存空间, requires_grad=False, 容易出错)

In [9]:
tensor_t = tensor_a.data
tensor_t[0] = -3.0
tensor_a, tensor_t, tensor_a == tensor_t, id(tensor_a), id(tensor_t)

(tensor([-3.,  2.,  3.], requires_grad=True),
 tensor([-3.,  2.,  3.]),
 tensor([True, True, True]),
 140300163941288,
 140300163961048)

In [10]:
tensor_t = tensor_a
tensor_t[0] = -4.0
tensor_a, tensor_t, tensor_a == tensor_t, id(tensor_a), id(tensor_t)

(tensor([-4.,  2.,  3.], grad_fn=<CopySlices>),
 tensor([-4.,  2.,  3.], grad_fn=<CopySlices>),
 tensor([True, True, True]),
 140300163941288,
 140300163941288)

## tensor.detach (会被自动求导系统追踪, 使用起来更安全)

In [11]:
tensor_t = tensor_a.detach()
tensor_t[0] = -2.0
tensor_a, tensor_t, tensor_a == tensor_t, id(tensor_a), id(tensor_t)

(tensor([-2.,  2.,  3.], grad_fn=<CopySlices>),
 tensor([-2.,  2.,  3.]),
 tensor([True, True, True]),
 140300163941288,
 140300163961480)

## tensor.data vs tensor.detch

In [12]:
tensor_a = torch.tensor([1, 2.0, 3.], requires_grad =True)
tensor_b = tensor_a.sigmoid()
tensor_c = tensor_b.data
tensor_c.zero_(), tensor_b, tensor_b.sum().backward(), tensor_a.grad # 不期望的结果(还不报错)

(tensor([0., 0., 0.]),
 tensor([0., 0., 0.], grad_fn=<SigmoidBackward>),
 None,
 tensor([0., 0., 0.]))

In [13]:
tensor_b = tensor_a.sigmoid()
tensor_c = tensor_b.detach()
tensor_c.zero_(), tensor_b, tensor_b.sum().backward(), tensor_a.grad # 会报错提示

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [3]], which is output 0 of SigmoidBackward, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

# 张量基本信息

In [ ]:
tensor = torch.rand((3,4))
print(tensor.type())   # Data type
print(tensor.size())   # Shape of the tensor. It is a subclass of Python tuple
print(tensor.dim())    # Number of dimensions.

# 数据类型转换

## 基础类型

In [ ]:
torch.set_default_tensor_type(torch.FloatTensor)
a = torch.randn(1, requires_grad=True)
b = a.cuda()
c = b.cpu()
d = c.int()
e = d.long()
a.type(), b.type(), c.type(), d.type(), e.type()

## torch.Tensor -> np.ndarray

In [ ]:
ndarray = tensor.cpu().numpy()
ndarray 

## np.ndarray -> torch.Tensor

In [ ]:
tensor = torch.from_numpy(ndarray).float()
# tensor = torch.from_numpy(ndarray.copy()).float()
tensor

## torch.item() (只适用于一个元素的情况)

a = torch.randn(1)
b = a.item()
a, b, type(b) # float

## torch.tolist() (多个元素转换为list)

In [ ]:
aa = torch.rand(3, 4)
bb = aa.cuda()
cc = aa.tolist()
dd = bb.tolist()
aa, bb, cc, dd, type(cc), type(dd) # list

## PIL.Image -> torch.Tensor.

In [ ]:
image = PIL.Image.open(os.path.join(k12ai_get_top_dir(), 'assets/bear.jpg'))

tensor1 = torch.from_numpy(np.asarray(image)).permute(2, 0, 1).float() / 255

tensor2 = torchvision.transforms.functional.to_tensor(image)

tensor3 = torchvision.transforms.ToTensor()(image)

In [ ]:
tensor1.shape, tensor2.shape, tensor2.shape, torch.equal(tensor1, tensor2), torch.equal(tensor1, tensor3)

## torch.Tensor -> PIL.Image.

In [ ]:
image1 = PIL.Image.fromarray(torch.clamp(tensor1 * 255, min=0, max=255
    ).byte().permute(1, 2, 0).cpu().numpy())

image2 = torchvision.transforms.functional.to_pil_image(tensor2)  # Equivalently way

image3 = torchvision.transforms.ToPILImage()(tensor3)

In [ ]:
display(image1)
display(image2)
display(image3)

# 从只包含一个元素的张量中提取值

In [ ]:
value = tensor[0][0].item()
value

# 张量形变

In [ ]:
tensor = torch.reshape(tensor, (4,3))
tensor

## 打乱顺序 ??

In [ ]:
tensor = tensor[torch.randperm(tensor.size(0))]  # Shuffle the first dimension
tensor

# 水平翻转

In [ ]:
# TODO 
# Assume tensor has shape N*D*H*W.
# tensor = tensor[:, :, :, torch.arange(tensor.size(3) - 1, -1, -1).long()]

# 复制张量

In [ ]:
# Operation                 |  New/Shared memory | Still in computation graph |
tensor.clone()            # |        New         |          Yes               |
tensor.detach()           # |      Shared        |          No                |
tensor.detach().clone()   # |        New         |          No                |

# 拼接张量

注意torch.cat和torch.stack的区别在于torch.cat沿着给定的维度拼接，而torch.stack会新增一维。
例如当参数是3个10×5的张量，torch.cat的结果是30×5的张量，而torch.stack的结果是3×10×5的张量。

In [ ]:
tensor1 = torch.rand((2, 3))
tensor2 = torch.rand((3, 3))

tensor = torch.cat((tensor1, tensor2), dim=0)
tensor1, tensor2, tensor

In [ ]:
tensor1 = torch.rand((2, 3))
tensor2 = torch.rand((2, 3))
tensor3 = torch.rand((2, 3))
tensor = torch.stack((tensor1, tensor2, tensor3), dim=0)
tensor.shape, tensor1.shape

# 将整数标记转换成独热（one-hot）编码

In [ ]:
# TODO
# one_hot = torch.zeros(100, 10).long()
# one_hot.scatter_(dim=1, index=torch.unsqueeze(tensor, dim=1), src=torch.ones(100, 10).long())

# 得到非零/零元素


In [ ]:
print(torch.nonzero(tensor))              # Index of non-zero elements
print(torch.nonzero(tensor == 0))          # Index of zero elements
print(torch.nonzero(tensor).size(0))       # Number of non-zero elements
print(torch.nonzero(tensor == 0).size(0))  # Number of zero elements

# Sort (同时返回对应的index)

In [ ]:
x = torch.randn(3,4)
x

In [ ]:
sorted, indices = torch.sort(x)
sorted, indices